In [1]:
# TOMA LOS KMLS NUEVO Y LOS AGREGA A SHP DE RECORRIDOS

In [18]:
import os
import re
import pandas as pd
import geopandas as gpd
from lxml import etree as ET
from shapely.geometry import LineString

import sys
sys.path.append('..')
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE

In [19]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [20]:
# RUTA ONEDRIVE
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

'G:\\OneDrive - Ingenio Azucarero Guabira S.A\\_DATOS_PYTHON'

In [21]:
# Convertir 'Flight Time' de milisegundos a minutos y segundos (MM:SS)
def convertir_tiempo(ms):
    try:
        ms = int(ms.strip())  # Convertir a entero y eliminar espacios extra
        segundos = ms // 1000  # Convertir milisegundos a segundos
        minutos = segundos // 60
        segundos = segundos % 60
        return f"{minutos}:{segundos:02d}"  # Formato MM:SS (ej. 9:35)
    except (ValueError, AttributeError):
        return None  # Si hay error, devolver None

def safe_float(value):
    """Convierte un valor a float, si es posible. Si es None o no numérico, devuelve None."""
    try:
        return float(value) if value is not None and value.strip() != "" else None
    except ValueError:
        return None

In [22]:
def convertir_kmls_a_shapefile(kml_folder, contenido):
    """
    Convierte varios archivos KML en un solo shapefile con proyección UTM zona 20S.
    Extrae valores de ExtendedData y los agrega como columnas.

    Parámetros:
    kml_folder (str): Carpeta que contiene los archivos KML.
    shapefile_path (str): Ruta donde se guardará el archivo shapefile combinado.
    """
    # Definir namespace de KML
    NAMESPACE = {"kml": "http://www.opengis.net/kml/2.2"}

    # Listas para almacenar los datos
    line_geometries = []
    filenames = []
    fechas = []
    horas = []
    ids = []
    
    aircraft_names = []
    flight_controller_ids = []
    pilot_names = []
    flight_times = []
    mode_selections = []
    heights = []
    route_spacings = []
    task_flight_speeds = []
    task_areas = []
    spray_amounts = []
    
    for filename in contenido:
        if filename.endswith('.kml'):
            kml_path = os.path.join(kml_folder, filename)
        
            try:
                with open(kml_path, 'r', encoding='utf-8') as file:
                    tree = ET.parse(file)
                    root = tree.getroot()
            except Exception as e:
                print(f"❌ Error al leer {filename}: {e}")
                continue
            
            # Extraer fecha, hora e ID del nombre del archivo
            # match = re.search(r'T\d+_(\d{4})(\d{2})(\d{2})(\d{2})(\d{2})(\d{2})_(R\d+)\.kml', filename)
            match = re.search(r'T\d+-\d+_(\d{4})(\d{2})(\d{2})(\d{2})(\d{2})(\d{2})_(R\d+)\.kml', filename)
            if not match:
                print(f"⚠ Advertencia: Nombre de archivo {filename} no coincide con el patrón esperado.")
                continue
            
            date = f"{match.group(1)}-{match.group(2)}-{match.group(3)}"
            time = f"{match.group(4)}:{match.group(5)}:{match.group(6)}"
            record_id = match.group(7)

            # Buscar Placemark en XML
            placemarks = root.findall('.//Placemark')
            if not placemarks:
                placemarks = root.findall('.//kml:Placemark', NAMESPACE)

            if not placemarks:
                print(f"⚠ Advertencia: No se encontraron <Placemark> en {filename}.")
                continue

            for placemark in placemarks:
                # Buscar LineString correctamente
                line_string = placemark.find('.//LineString')
                if line_string is None:
                    line_string = placemark.find('.//kml:LineString', NAMESPACE)
                
                if line_string is not None:
                    # Buscar Coordinates correctamente
                    coordinates = line_string.find('.//coordinates')
                    if coordinates is None:
                        coordinates = line_string.find('.//kml:coordinates', NAMESPACE)

                    if coordinates is not None and coordinates.text.strip():
                        coords = coordinates.text.strip().split()
                        points = [tuple(map(float, coord.split(',')[:2])) for coord in coords]  # Solo lat, lon
                        line_geometries.append(LineString(points))
                        fechas.append(date)
                        horas.append(time)
                        ids.append(record_id)
                        filenames.append(filename)

                        # Extraer datos de ExtendedData correctamente
                        extended_data = placemark.find('.//ExtendedData')
                        if extended_data is None:
                            extended_data = placemark.find('.//kml:ExtendedData', NAMESPACE)
                        
                        data_dict = {}
                        if extended_data is not None:
                            for data in extended_data.findall('.//Data') or extended_data.findall('.//kml:Data', NAMESPACE):
                                if data is None:
                                    continue  # Skip if there's no Data element

                                name = data.get('name')
                                value_element = data.find('.//kml:value', NAMESPACE)
                                if value_element is None:
                                    value_element = data.find('.//value')

                                value = value_element.text.strip() if value_element is not None and value_element.text else None
                                if name:
                                    data_dict[name] = value  # Store only if the name exists
                    
                        # Agregar valores de ExtendedData a las listas
                        aircraft_names.append(data_dict.get("Aircraft Name", None))
                        flight_controller_ids.append(data_dict.get("Flight Controller ID", None))
                        pilot_names.append(data_dict.get("Pilot Name", None))
                        flight_times.append(convertir_tiempo(data_dict.get("Flight Time", "0")))
                        mode_selections.append(data_dict.get("Mode Selection", None))
                        heights.append(data_dict.get("Height", None))
                        route_spacings.append(data_dict.get("Route Spacing", None))
                        
                        speed_ms = data_dict.get("Task Flight Speed")  # Obtener el valor si existe
                        if speed_ms is None or speed_ms.strip() == "":  # Manejar valores vacíos o None
                            speed_kmh = None
                        else:
                            speed_kmh = float(speed_ms) * 3.6 if speed_ms.replace('.', '', 1).isdigit() else None  # Convertir a km/h
                        task_flight_speeds.append(speed_kmh)
                        
                        task_areas.append(data_dict.get("Task Area", None))
                        spray_amounts.append(data_dict.get("Spray amount", None))

    if not line_geometries:
        print("❌ No se encontraron geometrías válidas en los archivos KML.")
        return

    # Crear un GeoDataFrame con todas las columnas
    gdf = gpd.GeoDataFrame({
        'filename': filenames,
        'fecha': fechas,
        'hora': horas,
        'id': ids,
        'aircraft_name': aircraft_names,
        'flight_controller_id': flight_controller_ids,
        'pilot_name': pilot_names,
        'flight_time': flight_times,
        'mode_selection': mode_selections,
        'height': heights,
        'route_spacing': route_spacings,
        'task_flight_speed': task_flight_speeds,
        'task_area': task_areas,
        'spray_amount': spray_amounts,
        'geometry': line_geometries
    }, crs='EPSG:4326')

    # Transformar a UTM zona 20S (EPSG:32720)
    gdf_utm = gdf.to_crs(epsg=32720)

    gdf_utm['spray_amount'] = gdf_utm['spray_amount'].astype(float) / 1000
    gdf_utm['height'] = gdf_utm['height'].astype(float)
    gdf_utm['route_spacing'] = gdf_utm['route_spacing'].astype(float)
    gdf_utm['task_area'] = gdf_utm['task_area'].astype(float)
    
    gdf_utm.rename(columns={
    'filename': 'file',
    'aircraft_name': 'drone',
    'flight_controller_id': 'ctrl_id',
    'pilot_name': 'pilot',
    'flight_time': 'fl_time',
    'mode_selection': 'mode',
    'height': 'height',
    'route_spacing': 'spacing',
    'task_flight_speed': 'fl_speed',
    'task_area': 'area',
    'spray_amount': 'spray' }, inplace=True)
    
    # Guardar como shapefile
    # gdf_utm.to_file(shapefile_path, driver='ESRI Shapefile')
    # print(f"✅ Archivo shapefile guardado en: {shapefile_path}")
    return gdf_utm

In [23]:
#leer base de datos de recorridos
path_recorridos = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\SHP\RECORRIDOS.shp'
gdf_recorridos = gpd.read_file(path_recorridos)
gdf_recorridos.head(3)

,file,fecha,hora,id,drone,ctrl_id,pilot,fl_time,mode,height,spacing,fl_speed,area,spray,idd,geometry
0,T40_20250307131940_R5630753528.kml,2025-03-07,13:19:40,R5630753528,T40,1581F574B238900100GF,GABRIEL PENA,9:26,Auto,5.0,9.64,29.880001,3.350000,33.825,1,"LINESTRING (503877.967 8094579.033, 503877.974..."
1,T40_20250307133501_R6431655408.kml,2025-03-07,13:35:01,R6431655408,T40,1581F574B238900100GF,GABRIEL PENA,9:46,Auto,5.0,9.64,29.880001,3.548667,35.917,1,"LINESTRING (503876.695 8094578.178, 503876.695..."
2,T40_20250307134823_R6551790690.kml,2025-03-07,13:48:23,R6551790690,T40,1581F574B238900100GF,GABRIEL PENA,9:13,Auto,4.4,9.64,29.880001,3.148667,31.913,1,"LINESTRING (503881.318 8094576.323, 503881.287..."


In [24]:
path_kml = unidad + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Pulverizacion\2025\KML_RECORRIDOS'
contenido = os.listdir(path_kml)
len(contenido)

1478

In [25]:
# Filtrar la lista 'contenido' para excluir elementos que están en la columna 'nombre' del GeoDataFrame
contenido_filtrado = [item for item in contenido if item not in gdf_recorridos['file'].values]
len(contenido_filtrado)

0

In [26]:
gdf_nuevos_recorridos = convertir_kmls_a_shapefile(path_kml, contenido_filtrado)

❌ No se encontraron geometrías válidas en los archivos KML.


In [27]:
gdf_nuevos_recorridos['idd'] = 0

TypeError: 'NoneType' object does not support item assignment

In [11]:
len(gdf_nuevos_recorridos)

90

In [13]:
gdf_combined = gpd.GeoDataFrame(pd.concat([gdf_recorridos, gdf_nuevos_recorridos], ignore_index=True))

In [14]:
gdf_combined

,file,fecha,hora,id,drone,ctrl_id,pilot,fl_time,mode,height,spacing,fl_speed,area,spray,idd,geometry
0,T40_20250307131940_R5630753528.kml,2025-03-07,13:19:40,R5630753528,T40,1581F574B238900100GF,GABRIEL PENA,9:26,Auto,5.0,9.64,29.880001,3.350000,33.825,1,"LINESTRING (503877.967 8094579.033, 503877.974..."
1,T40_20250307133501_R6431655408.kml,2025-03-07,13:35:01,R6431655408,T40,1581F574B238900100GF,GABRIEL PENA,9:46,Auto,5.0,9.64,29.880001,3.548667,35.917,1,"LINESTRING (503876.695 8094578.178, 503876.695..."
2,T40_20250307134823_R6551790690.kml,2025-03-07,13:48:23,R6551790690,T40,1581F574B238900100GF,GABRIEL PENA,9:13,Auto,4.4,9.64,29.880001,3.148667,31.913,1,"LINESTRING (503881.318 8094576.323, 503881.287..."
3,T40_20250307140126_R6611858331.kml,2025-03-07,14:01:26,R6611858331,T40,1581F574B238900100GF,GABRIEL PENA,9:23,Auto,4.4,9.64,29.880001,3.354000,34.072,1,"LINESTRING (503880.632 8094579.144, 503880.631..."
4,T40_20250307141430_R6832106348.kml,2025-03-07,14:14:30,R6832106348,T40,1581F574B238900100GF,GABRIEL PENA,10:48,Auto,4.4,9.64,29.880001,3.429333,34.590,1,"LINESTRING (503880.530 8094579.078, 503880.530..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1473,T50-1_20250813160029_R5337499412.kml,2025-08-13,16:00:29,R5337499412,T50-1,1581F6BUB24630011R08,Eduardo Reyes,10:00,Auto,4.0,8.00,28.080001,2.914000,29.400,0,"LINESTRING (483884.961 8101276.880, 483884.961..."
1474,T50-1_20250813161457_R2862026203.kml,2025-08-13,16:14:57,R2862026203,T50-1,1581F6BUB24630011R08,Eduardo Reyes,7:36,Auto,4.0,8.00,28.080001,1.651333,16.776,0,"LINESTRING (483884.436 8101276.648, 483884.436..."
1475,T50-1_20250813162233_R2902071297.kml,2025-08-13,16:22:33,R2902071297,T50-1,1581F6BUB24630011R08,Eduardo Reyes,2:17,Auto,4.0,8.00,28.080001,0.342000,3.442,0,"LINESTRING (484046.454 8101100.388, 484046.457..."
1476,T50-1_20250813162729_R6087118331.kml,2025-08-13,16:27:29,R6087118331,T50-1,1581F6BUB24630011R08,Eduardo Reyes,10:01,Auto,4.0,8.00,28.080001,2.644000,26.667,0,"LINESTRING (483884.756 8101276.276, 484020.058..."


In [15]:
gdf_combined.dtypes

file          object
fecha         object
hora          object
id            object
drone         object
ctrl_id       object
pilot         object
fl_time       object
mode          object
height       float64
spacing      float64
fl_speed     float64
area         float64
spray        float64
idd            int64
geometry    geometry
dtype: object

In [16]:
gdf_combined.to_file(path_recorridos, driver='ESRI Shapefile')